# Computes the Musical "Circle of Fifths" and Deposits it into Neo4j

Computes the "Circle of Fifths" from Western music theory. Keys and modes are given in DJ notation, where each relationship "KEY_AND_MODE_PATH" between keys ensures that the a song played in source node's key will prove harmonically aligned with the next song played in receiving node's key. This information enables DJs to mix songs live.

I'm working on using AI to generate better playlists. With this content loaded into Neo4j I can trace acceptable paths through possible songs (Note:  The songs and their attributes are not loaded yet).

## Load useful libraries

In [ ]:
import pandas as pd
from neo4j import GraphDatabase

## User settings

In [ ]:
graph_database_uri = 'neo4j://127.0.0.1:7687'
graph_database_username = '*****'
graph_database_password = '*****'
graph_database_name = '*****'

## Define a class to hold the "circle of fifths" from western music theory

In [ ]:
class circle_of_fifths():

    #
    # constructor
    #
    def __init__(
        self,
        chromatic_scale = ['C', 'C#/Db', 'D', 'D#/Eb', 'E', 'F', 'F#/Gb', 'G', 'G#/Ab', 'A', 'A#/Bb', 'B'],
    ):
        
        # initialize
        self.chromatic_scale = chromatic_scale
        self.chromatic_scale_base_zero_indices = list(range(0, len(self.chromatic_scale)))
        
        # compute circle of fifths
        self.compute_circle_of_fifths_from_chromatic_scale()

        # shift root notes to match the Camelot EasyMix wheel
        self.shift_root_notes_to_match_Camelot()

        # calculate wheel
        self.create_Camelot_wheel()
    
    #
    # An interval of seven on a chromatic scale equals an interval of a fifth on an octave scale
    #
    def compute_circle_of_fifths_from_chromatic_scale(self):
        self.fifths_list = []
        for j in range(0, len(self.chromatic_scale) * 7, 7):
            self.fifths_list.append(self.chromatic_scale[j % 12])
    
    #
    # Align with Camelot EasyMix notation
    #
    def shift_root_notes_to_match_Camelot(self):
        self.chromatic_camelot_major = self.fifths_list[-7:]
        self.chromatic_camelot_major.extend(self.fifths_list[0:-7])
        self.chromatic_camelot_minor = self.fifths_list[-4:]
        self.chromatic_camelot_minor.extend(self.fifths_list[0:-4])
        
    #
    # Create virtual representation of the Camelot EasyMix wheel
    #
    def create_Camelot_wheel(self):

        all_keys_as_list = []
        key_mode_dict = {}

        for mode, mode_switch in zip(['A', 'B'], ['B', 'A']):
            for k in self.chromatic_scale_base_zero_indices:
    
                k_down = (self.chromatic_scale_base_zero_indices[k] - 1) % 12
                k_same = (self.chromatic_scale_base_zero_indices[k]) % 12       # this could easily be cut, but I thought coding it up would improve review understanding
                k_up = (self.chromatic_scale_base_zero_indices[k] + 1) % 12
    
                k_down_for_display = str(k_down + 1) + mode
                k_same_for_display = str(k_same + 1) + mode
                k_same_for_display_with_mode_switch = str(k_same + 1) + mode_switch
                k_up_for_display = str(k_up + 1) + mode
 
                key_mode_dict[k_same_for_display] = {
                    'same' : k_same_for_display,
                    'same_with_mode_switch' : k_same_for_display_with_mode_switch,
                    'down' : k_down_for_display,
                    'up' : k_up_for_display,
                    'key_numeric' : k + 1,
                    'mode' : k_same_for_display[-1],
                }
    
                if mode == 'A':
                    key_mode_dict[k_same_for_display]['key_pitch_class'] = self.chromatic_camelot_minor[k]
                elif mode == 'B':
                    key_mode_dict[k_same_for_display]['key_pitch_class'] = self.chromatic_camelot_major[k]
            
                all_keys_as_list.append(key_mode_dict[k_same_for_display])

        self.df = pd.DataFrame(all_keys_as_list)
        
        
    def prepare_for_Neo4j_load(self):
        self.cmd_list = []

        for i, row in self.df.iterrows():
    
            self.cmd_list.append(("""MERGE (m : CAMELOT_KEY_AND_MODE {id : $name_i, name : $name_i}) RETURN m""", (row['same'], row['same'])))
    
            self.cmd_list.append(
                ("""
                MATCH (i : CAMELOT_KEY_AND_MODE {id : $name_i, name : $name_i}), (j : CAMELOT_KEY_AND_MODE {id : $name_j, name : $name_j})
                MERGE (i)-[r:KEY_AND_MODE_PATH {method : 'mode_change'}]->(j) RETURN i, j""", (row['same'], row['same_with_mode_switch'])))
        
            self.cmd_list.append(("""
                MATCH (i : CAMELOT_KEY_AND_MODE {id : $name_i, name : $name_i}), (j : CAMELOT_KEY_AND_MODE {id : $name_j, name : $name_j})
                MERGE (j)-[r:KEY_AND_MODE_PATH {method : 'mode change'}]->(i) RETURN j, i""", (row['same'], row['same_with_mode_switch'])))
    
            self.cmd_list.append(("""
                MATCH (i : CAMELOT_KEY_AND_MODE {id : $name_i, name : $name_i}), (j : CAMELOT_KEY_AND_MODE {id : $name_j, name : $name_j})
                MERGE (i)-[r:KEY_AND_MODE_PATH {method : 'down'}]->(j) RETURN i, j""", (row['same'], row['down'])))
    
            self.cmd_list.append(("""
                MATCH (i : CAMELOT_KEY_AND_MODE {id : $name_i, name : $name_i}), (j : CAMELOT_KEY_AND_MODE {id : $name_j, name : $name_j})
                MERGE (i)-[r:KEY_AND_MODE_PATH {method : 'up'}]->(j) RETURN j, i""", (row['same'], row['up'])))
        
            self.cmd_list.append(("""
                MATCH (i : CAMELOT_KEY_AND_MODE {id : $name_i, name : $name_i}), (j : CAMELOT_KEY_AND_MODE {id : $name_j, name : $name_j})
                MERGE (i)-[r:KEY_AND_MODE_PATH {method : 'stay'}]->(j) RETURN i, j""", (row['same'], row['same'])))

## Instantiate an object of this class

In [ ]:
cf = circle_of_fifths()
cf.prepare_for_Neo4j_load()

## Connect to Neo4j

In [ ]:
with GraphDatabase.driver(graph_database_uri, auth = (graph_database_username, graph_database_password)) as driver:
    driver.verify_connectivity()

## Define a transaction for circle of fifths' elements

In [ ]:
def circle_of_fifths_tx(tx, cmd):
    values_tuple = cmd[1]
    name_i = values_tuple[0]
    name_j = values_tuple[1]
    result = tx.run(cmd[0], name_i = values_tuple[0], name_j = values_tuple[1])

## Main

In [ ]:
def main():
    with GraphDatabase.driver(graph_database_uri, auth=(graph_database_username, graph_database_password)) as driver:
        with driver.session(database = 'neo4j') as session:
            for cmd in cf.cmd_list:
                session.execute_write(circle_of_fifths_tx, cmd)

if __name__ == '__main__':
    main()